In [ ]:
import os
import torch
import json
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from PIL import Image
import cv2

# ========== FIX 1: Extract lib files ==========
!unzip -q /kaggle/input/tgs-salt-identification-challenge/flamingo.zip -d /kaggle/working/
!mv /kaggle/working/flamingo/lib /kaggle/working/lib

# ========== FIX 2: Add to Python path ==========
import sys
sys.path.insert(0, '/kaggle/working/lib')

# ========== Now the imports will work ==========
from train_utils import get_model
from postprocess import zero_masks_inplace

# Step 1: Setup environment
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = json.load(open('/kaggle/input/tgs-salt-identification-challenge/infallible_lamport.json'))
snapshot = torch.load('/kaggle/input/tgs-salt-identification-challenge/Oct09_23_17_wider_unet_224pad_medium_infallible_lamport_val_lb.pth', map_location=device)

# Step 2: Prepare the model
model = get_model(config['model'],
                  num_classes=config['num_classes'],
                  num_channels=1,
                  pretrained=False).to(device)

model.load_state_dict(snapshot['model'])
model.eval()

# Step 3: Load test data
test_folder = '/kaggle/input/tgs-salt-identification-challenge/test'
test_images = sorted([os.path.join(test_folder, f) for f in os.listdir(test_folder) if f.endswith('.png')])
test_ids = [os.path.basename(f).replace('.png', '') for f in test_images]

# Step 4: Define the test dataset
class SaltTestDataset(Dataset):
    def __init__(self, image_paths):
        self.paths = image_paths
        self.ids = [os.path.basename(p).replace('.png', '') for p in self.paths]

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image = Image.open(self.paths[idx])
        image = np.array(image, dtype=np.float32) / 255.0
        image = np.expand_dims(image, axis=0)  # shape: (1, 101, 101)
        return torch.tensor(image), self.ids[idx]

# Step 5: Prepare DataLoader
dataset = SaltTestDataset(test_images)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

# Step 6: Generate predictions
pred_masks = []
image_ids = []

with torch.no_grad():
    for images, ids in tqdm(loader, desc="Predicting"):
        images = images.to(device)
        output = model(images)

        # Process the model output
        masks = output.sigmoid().cpu().numpy()
        masks = (masks > 0.45).astype(np.uint8)  # Apply threshold

        for mask in masks:
            mask = np.squeeze(mask)
            pred_masks.append(mask)

        image_ids.extend(ids)

# Step 7: Run-Length Encoding (RLE) for submission
def rle_encode(mask):
    pixels = mask.T.flatten()  # Fix: Use Fortran-order flattening
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(map(str, runs)) if len(runs) else ''

# Step 8: Format submission
encoded_masks = [rle_encode(mask) for mask in pred_masks]
submission_df = pd.DataFrame({
    'id': image_ids,
    'rle_mask': encoded_masks
})


# Step 9: Save the submission file
submission_df.to_csv('/kaggle/working/submission.csv.gz', index=False, compression='gzip')

print("✅ Submission saved to /kaggle/working/submission.csv.gz")